In [1]:
import pandas as pd
from datetime import datetime


In [2]:
def endprice_string_to_number(input_end_price):
    price =""
    for index in range(len(input_end_price)):
        if(input_end_price[index].isnumeric()):
            price = price+input_end_price[index]
    price = int(price)
    return price
def KvMPrice_string_to_number(input_end_price):
    price = ""
    if input_end_price is None:
        return None
    else:
        parts = input_end_price.split()
        for each_part in parts:
            if(each_part.isnumeric()):
                price = price + each_part
            
        price = int(price)
        return price
def fee_string_to_number(input_fee):
    price =""
    for index in range(len(input_fee)):
        if(input_fee[index].isnumeric()):
            price = price+input_fee[index]
    price = int(price)
    return price
def selling_date_format(selling_date_input):
    selling_date = selling_date_input[4:]
    date = datetime.strptime(selling_date, '%d%B%Y')
    formatted_date = date.strftime('%Y-%m-%d')
    return(formatted_date)

def city_and_district_format_return_city(city_and_district):
    separated = city_and_district.strip(" ").split("\n")
    return((separated[1]))

def locate_city(each_house):
    div_element = each_house.find("div")
    text_content = list(div_element.stripped_strings)
    return((text_content[5]).split("\n"))
    

In [3]:
#scraping single house block data
from bs4 import  BeautifulSoup
import requests
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
web_link = r"https://www.hemnet.se/salda/bostader?location_ids%5B%5D=%27+Town+%27&page=1&sold_age=10m"
html_text = requests.get(web_link, headers=headers).text
soup = BeautifulSoup(html_text,"lxml")
house_block =soup.find("li",class_="sold-results__normal-hit")
#print(house_block)

house_name = house_block.find("h2",class_ = "sold-property-listing__heading qa-selling-price-title hcl-card__title").text.replace(" ","") # Obtain name of our house
house_name =house_name.strip() #to prevent the output to go to a new line
print("Name of the house : ",house_name)

house_size_and_number_of_rooms = (house_block.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text) if (house_block.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text.replace(" ","")) else None
#house_block.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text.replace(" ","")
size_and_rooms_list = house_size_and_number_of_rooms.split()
size_of_room = float(size_and_rooms_list[0].replace(",","."))


number_of_rooms = (size_and_rooms_list[-2])
print("Size of the room :",size_of_room,"\nNumber of rooms :",number_of_rooms)

house_fee = (house_block.find("span", class_="sold-property-listing__fee").text.replace(" ", "").strip()) if (house_block.find("span", class_="sold-property-listing__fee")) else None

print("House fee: ",(house_fee))

house_end_price = endprice_string_to_number((house_block.find("span",class_= "hcl-text hcl-text--medium").text.replace(" ","")).strip()) if (house_block.find("span",class_= "hcl-text hcl-text--medium").text.replace(" ","")) else None
#house_block.find("span",class_= "hcl-text hcl-text--medium").text.replace(" ","")
print("End Price",house_end_price)

house_KvMPrice = (house_block.find("div", class_="sold-property-listing__price-per-m2").text.replace(" ", "").strip()) if (house_block.find("div", class_="sold-property-listing__price-per-m2")) else None
print("Price per sq m: ",house_KvMPrice)

house_SaleDate = (house_block.find("span",class_="hcl-label hcl-label--state hcl-label--sold-at").text.replace(" ","")).strip() if (house_block.find("span",class_="hcl-label hcl-label--state hcl-label--sold-at").text.replace(" ","")) else None
#house_block.find("span",class_="hcl-label hcl-label--state hcl-label--sold-at").text.replace(" ","")
print("Date of selling : ",house_SaleDate)

house_city_district = (house_block.find("span",class_="property-icon property-icon--result").text.replace(" ","")).strip() if (house_block.find("span",class_="property-icon property-icon--result").text.replace(" ","")) else None
print("City : ",house_city_district)

house_type_of_property = (house_block.find("span",class_= "property-icon property-icon--result").text.replace(" ","")) if(house_block.find("span",class_= "property-icon property-icon--result").text.replace(" ","")) else None
print("Type of property : ",house_type_of_property)

Name of the house :  Björkbacksvägen53
Size of the room : 89.5 
Number of rooms : 3
House fee:  None
End Price 2200000
Price per sq m:  24 581 kr/m²
Date of selling :  Såld4oktober2023
City :  LägenhetLägenhet
Type of property :  LägenhetLägenhet



In [4]:
house_block_all =soup.find_all("li",class_="sold-results__normal-hit")
number_of_houses = 0
#for each_house in house_block_all:
    #print(each_house)
house_data_data_frame = pd.DataFrame(columns=["Name","Size","Room","Fee","Endprice","KvMPrice","Saledate","City","District","TypeOfProperty"])


In [5]:
'''
i = 0 # index iteration
for each_house in house_block_all:
      house_data_data_frame.loc[i,"Name"] =(each_house.find("h2",class_ = "sold-property-listing__heading qa-selling-price-title hcl-card__title").text.replace(" ","")).strip()
      
      house_size_and_number_of_rooms = (each_house.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text.replace(" ","")) if (house_block.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text.replace(" ","")) else None
      size_and_rooms_list = house_size_and_number_of_rooms.split()
      size_of_room = float(size_and_rooms_list[0].replace(",","."))
      house_data_data_frame.loc[i,"Size"] = size_of_room
      house_data_data_frame.loc[i,"Room"] = (size_and_rooms_list[-2])
      house_data_data_frame.loc[i,"Fee"] =  fee_string_to_number(each_house.find("div", class_="sold-property-listing__fee").text.replace(" ", "").strip()) if (each_house.find("div", class_="sold-property-listing__fee")) else None
      house_data_data_frame.loc[i,"Endprice"] =  endprice_string_to_number((each_house.find("span",class_= "hcl-text hcl-text--medium").text.replace(" ","")).strip()) if (house_block.find("span",class_= "hcl-text hcl-text--medium").text.replace(" ","")) else None
      house_data_data_frame.loc[i,"KvMPrice"] = KvMPrice_string_to_number(((each_house.find("div", class_="sold-property-listing__price-per-m2"))).text.replace(" ", "").strip()) if (each_house.find("div", class_="sold-property-listing__price-per-m2") is not None)  else None
      house_data_data_frame.loc[i,"Saledate"] = selling_date_format((each_house.find("span",class_="hcl-label hcl-label--state hcl-label--sold-at").text.replace(" ","")).strip()) if (house_block.find("span",class_="hcl-label hcl-label--state hcl-label--sold-at").text.replace(" ","")) else None
      house_data_data_frame.loc[i,"City"] = ((each_house.find("span", class_="property-icon property-icon--result").next_sibling.replace(" ","")).split(","))[1].strip()
      house_data_data_frame.loc[i,"District"] = ((each_house.find("span", class_="property-icon property-icon--result").next_sibling.replace(" ","")).split(","))[0].strip()
      house_data_data_frame.loc[i,"TypeOfProperty"] = each_house.find("span", class_="property-icon property-icon--result").find("title").text.replace(" ","").strip()


      i = i+1   
'''

'\ni = 0 # index iteration\nfor each_house in house_block_all:\n      house_data_data_frame.loc[i,"Name"] =(each_house.find("h2",class_ = "sold-property-listing__heading qa-selling-price-title hcl-card__title").text.replace(" ","")).strip()\n      \n      house_size_and_number_of_rooms = (each_house.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text.replace(" ","")) if (house_block.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text.replace(" ","")) else None\n      size_and_rooms_list = house_size_and_number_of_rooms.split()\n      size_of_room = float(size_and_rooms_list[0].replace(",","."))\n      house_data_data_frame.loc[i,"Size"] = size_of_room\n      house_data_data_frame.loc[i,"Room"] = (size_and_rooms_list[-2])\n      house_data_data_frame.loc[i,"Fee"] =  fee_string_to_number(each_house.find("div", class_="sold-property-listing__fee").text.replace(" ", "").strip()) if (each_house.find("div", class_="s

In [7]:
website_part_1 = "https://www.hemnet.se/salda/bostader?location_ids%5B%5D=%27+Town+%27&page=" 
website_part_2 = "&sold_age=10m"
house_data_data_frame = pd.DataFrame(columns=["Name","Size","Room","Fee","Endprice","KvMPrice","Saledate","City","District","TypeOfProperty"])
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}
i=0
for j in range (1,51):
    full_link = website_part_1 + str(j) + website_part_2
    
    web_link = full_link
    html_text = requests.get(web_link, headers=headers).text
    soup = BeautifulSoup(html_text,"lxml")
    house_block_all =soup.find_all("li",class_="sold-results__normal-hit")
    for each_house in house_block_all:
        try:
            house_data_data_frame.loc[i,"Name"] =(each_house.find("h2",class_ = "sold-property-listing__heading qa-selling-price-title hcl-card__title").text).strip()
            house_size_and_number_of_rooms = (each_house.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text.replace(" ","")) if (house_block.find("div",class_ = "sold-property-listing__subheading sold-property-listing__area").text.replace(" ","")) else None
            size_and_rooms_list = house_size_and_number_of_rooms.split()
            size_of_room = float(size_and_rooms_list[0].replace(",","."))
            house_data_data_frame.loc[i,"Size"] = size_of_room
            house_data_data_frame.loc[i,"Room"] = (size_and_rooms_list[-2])
            house_data_data_frame.loc[i,"Fee"] =  fee_string_to_number(each_house.find("div", class_="sold-property-listing__fee").text.replace(" ", "").strip()) if (each_house.find("div", class_="sold-property-listing__fee")) else None
            house_data_data_frame.loc[i,"Endprice"] =  endprice_string_to_number((each_house.find("span",class_= "hcl-text hcl-text--medium").text.replace(" ","")).strip()) if (house_block.find("span",class_= "hcl-text hcl-text--medium").text.replace(" ","")) else None
            house_data_data_frame.loc[i,"KvMPrice"] = KvMPrice_string_to_number(((each_house.find("div", class_="sold-property-listing__price-per-m2"))).text.replace(" ", "").strip()) if (each_house.find("div", class_="sold-property-listing__price-per-m2") is not None)  else None
            house_data_data_frame.loc[i,"Saledate"] = selling_date_format((each_house.find("span",class_="hcl-label hcl-label--state hcl-label--sold-at").text.replace(" ","")).strip()) if (house_block.find("span",class_="hcl-label hcl-label--state hcl-label--sold-at").text.replace(" ","")) else None
            house_data_data_frame.loc[i,"City"] = ((each_house.find("span", class_="property-icon property-icon--result").next_sibling)).split(",")[1].strip()
            house_data_data_frame.loc[i,"District"] = ((each_house.find("span", class_="property-icon property-icon--result").next_sibling).split(","))[0].strip()
            house_data_data_frame.loc[i,"TypeOfProperty"] = each_house.find("span", class_="property-icon property-icon--result").find("title").text.strip() 
        except:
            #print("i ,j= ",i,j)
            pass
        i = i+1   

    

In [8]:
path = r"C:\Users\akash\Documents\AI Sweden\Learning and Resources\Week 2\Hemnet_scraped_data.csv"
house_data_data_frame.to_csv(path, index=False,encoding='latin')


The above part of the program was intended to scrap the data from hemnet. However there might be certain entries in the csv file which look absurd as i scraped the data from one website and there after applied it to all the other pages which may have slight differences in the way they were formatted.

The next part of the program focusses on the next assigned work

Also the next part of the program is designed in such a manner such that only using the data from the csv one can work with it without implementing the code above and thus save time.

In [10]:
import pandas as pd
#path = r"C:\Users\akash\Documents\AI Sweden\Learning and Resources\Week 2\Hemnet_scraped_data.csv" Also use ,encoding="latin" here
#path = r"C:\Users\akash\Documents\AI Sweden\Learning and Resources\Week 2\House_data_2022_not_scraped_by_me.csv"
path = r"C:\Users\akash\Documents\AI Sweden\Learning and Resources\Week 2\Hemnet_scraped_data.csv"
house_data_frame = pd.read_csv(path,encoding="latin")

In [11]:
'''  
Load the csv file “housedata.csv” into the notebook with the help of pandas. Take out all the names of the houses that we don't have size values for, add them to an array.
In some cases the same housing or similar have been sold multiple times during the data timeframe, this will be able to fill in some of the gaps in the size data. Use your knowledge of the address and fix the size in the missing cases. 
If the sizes can not be fixed, remove the value altogether.
Do the same process for the missing rooms.
Convert the room and size variables from strings to ints.
Check if there are rows with more rooms than square meter, if so is the case switch place on these values.
Replace all the question marks with zeros, and do the same for NaN values.
Calculate the square meter price by dividing the selling price by the housing size.


'''

"  \nLoad the csv file “housedata.csv” into the notebook with the help of pandas. Take out all the names of the houses that we don't have size values for, add them to an array.\nIn some cases the same housing or similar have been sold multiple times during the data timeframe, this will be able to fill in some of the gaps in the size data. Use your knowledge of the address and fix the size in the missing cases. \nIf the sizes can not be fixed, remove the value altogether.\nDo the same process for the missing rooms.\nConvert the room and size variables from strings to ints.\nCheck if there are rows with more rooms than square meter, if so is the case switch place on these values.\nReplace all the question marks with zeros, and do the same for NaN values.\nCalculate the square meter price by dividing the selling price by the housing size.\n\n\n"

In [12]:
#Load the csv file “housedata.csv” into the notebook with the help of pandas. Take out all the names of the houses that we don't have size values for, add them to an array.
name_of_house_with_no_size = []
name_of_house_no_room_size = []
for i in range(len(house_data_frame)):
    if(pd.isnull(house_data_frame.at[i,"Size"]) == True):
        name_of_house_with_no_size.append(house_data_frame.loc[i,"Name"])
    if(pd.isnull(house_data_frame.at[i,"Room"]) == True):
        name_of_house_no_room_size.append(house_data_frame.loc[i,"Name"])
        
print(name_of_house_with_no_size)
print(name_of_house_no_room_size)


['Trulstorp 2:19', 'Sibbalt 1', 'Bältebergavägen 574', 'Snaversrud 309', 'Norrlanda Bringes Skifte 2 & 3', 'Grönvägen 50A', 'Torp i  Trönö med jakträtt', 'Skellefteå-Brännvattnet 4:2', 'Sibbarp 1:8', 'Hägerstensvägen 124 B', 'Skogså 219C']
['Trulstorp 2:19', 'Sibbalt 1', 'Bältebergavägen 574', 'Snaversrud 309', 'Norrlanda Bringes Skifte 2 & 3', 'Grönvägen 50A', 'Torp i  Trönö med jakträtt', 'Skellefteå-Brännvattnet 4:2', 'Sibbarp 1:8', 'Hägerstensvägen 124 B', 'Skogså 219C']


In [13]:
#In some cases the same housing or similar have been sold multiple times during the data timeframe, this will be able to fill in some of the gaps in the size data.
#Use your knowledge of the address and fix the size in the missing cases.
#If the sizes can not be fixed, remove the value altogether.
#Do the same process for the missing rooms
def search_housing_same_name(name_of_house):
    size_of_room = None
    number_of_rooms = None
    for i in range(0,len(house_data_frame)-1):
        if house_data_frame.loc[i, "Name"] == name_of_house:
            if pd.notna(house_data_frame.loc[i, "Size"]):
                size_of_room = (house_data_frame.loc[i, "Size"])
            if pd.notna(house_data_frame.loc[i, "Room"]):
                number_of_rooms = (house_data_frame.loc[i, "Room"])
                
    return size_of_room, number_of_rooms
for i in name_of_house_with_no_size:
    size_of_room, number_of_rooms = search_housing_same_name(i)
    house_data_frame.loc[house_data_frame['Name'] == i, 'Size'] = size_of_room
    house_data_frame.loc[house_data_frame['Name'] == i, 'Room'] = number_of_rooms
house_data_frame =house_data_frame.dropna(subset=["Size","Room"])


In [14]:
#Convert the room and size variables from strings to ints.
house_data_frame['Room'] = house_data_frame['Room'].replace(',','.',regex=True)
house_data_frame['Size'] = house_data_frame['Size'].replace(',','.',regex=True)
house_data_frame = house_data_frame.astype({"Size": float,"Room" : float})

In [15]:
#Check if there are rows with more rooms than square meter, if so is the case switch place on these values.
#Replace all the question marks with zeros, and do the same for NaN values.
for i in house_data_frame.index:
    if(house_data_frame.loc[i,"Room"] > house_data_frame.loc[i,"Size"]):
        house_data_frame.loc[i,"Room"] , house_data_frame.loc[i,"Size"] = house_data_frame.loc[i,"Size"] , house_data_frame.loc[i,"Room"]
house_data_frame = house_data_frame.replace(r"\?","0", regex=True)
house_data_frame = house_data_frame.fillna(0)

In [16]:
#Calculate the square meter price by dividing the selling price by the housing size.
house_data_frame["Square_meter_price"] = round((house_data_frame["KvMPrice"] / house_data_frame["Size"]),2)


In [17]:
''' 
Districts and housing types
The district's data are a real mess, some realtors write with capital letters, some without, some with two neighboring districts split by a ‘“/” or a “-”. This needs to be fixed!
fix all the districts to lowercase letters, except the first.
remove the double districts and use the first.
remove all the districts with postcodes; if you want, you can replace them with the names.
The same applies to type of properties. They are coupled together with “/” or having longer than needed type names.         
fix all the names that contains “rätt“ to be “Bostadsrättslägenhet”
fix all the names that contains “Par/Rad/Kedjehus“ to be “Radhus”
fix all the names that contains “illa“ to be “Villa”
fix all the names that contains “ård“ to be “Gård”
fix all the names that contains “hus“ to be “Fritidshus”
All the rest should be “other”
Save the data as “housing_clean.csv”.
Do the same for the 2022 data file

'''


" \nDistricts and housing types\nThe district's data are a real mess, some realtors write with capital letters, some without, some with two neighboring districts split by a ‘“/” or a “-”. This needs to be fixed!\nfix all the districts to lowercase letters, except the first.\nremove the double districts and use the first.\nremove all the districts with postcodes; if you want, you can replace them with the names.\nThe same applies to type of properties. They are coupled together with “/” or having longer than needed type names.         \nfix all the names that contains “rätt“ to be “Bostadsrättslägenhet”\nfix all the names that contains “Par/Rad/Kedjehus“ to be “Radhus”\nfix all the names that contains “illa“ to be “Villa”\nfix all the names that contains “ård“ to be “Gård”\nfix all the names that contains “hus“ to be “Fritidshus”\nAll the rest should be “other”\nSave the data as “housing_clean.csv”.\nDo the same for the 2022 data file\n\n"

In [18]:
#fix all the districts to lowercase letters, except the first.
#remove the double districts and use the first

house_data_frame["District"] = (house_data_frame["District"].str.capitalize())
for i in house_data_frame.index:
    if( (pd.notna(house_data_frame.loc[i,"District"]))) : 
        if( ("/" in house_data_frame.loc[i,"District"]) == True ):
            house_data_frame.loc[i,"District"] = (house_data_frame.loc[i,"District"]).split("/")[0]
        if(("-" in house_data_frame.loc[i,"District"]) == True):    
             house_data_frame.loc[i,"District"] = (house_data_frame.loc[i,"District"].split("-"))[0]

#remove all the districts with postcodes; if you want, you can replace them with the names. -not done as none were found but it can be added as a caluse later on        

In [19]:
#property fixture
#fix all the names that contains “rätt“ to be “Bostadsrättslägenhet”
#fix all the names that contains “Par/Rad/Kedjehus“ to be “Radhus”
#fix all the names that contains “illa“ to be “Villa”
#fix all the names that contains “ård“ to be “Gård”
#fix all the names that contains “hus“ to be “Fritidshus”
#All the rest should be “other”
#( (pd.notna(house_data_frame.loc[i,"TypeOfProperty"])) & type(house_data_frame.loc[i,"TypeOfProperty"]) ==str ) testing purpose
for i in house_data_frame.index:
    if (isinstance(house_data_frame.loc[i,"TypeOfProperty"],str)) : 
        
        if( ("rätt" in house_data_frame.loc[i,"TypeOfProperty"]) == True ):
            house_data_frame.loc[i,"TypeOfProperty"] = "Bostadsrättslägenhet"
        elif( ("Par" in house_data_frame.loc[i,"TypeOfProperty"]) == True ):
            house_data_frame.loc[i,"TypeOfProperty"] = "Radhus"
        elif( ("Rad" in house_data_frame.loc[i,"TypeOfProperty"]) == True ):
            house_data_frame.loc[i,"TypeOfProperty"] = "Radhus"
        elif( ("Kedjehus" in house_data_frame.loc[i,"TypeOfProperty"]) == True ):
            house_data_frame.loc[i,"TypeOfProperty"] = "Radhus"
        elif( ("illa" in house_data_frame.loc[i,"TypeOfProperty"]) == True ):
            house_data_frame.loc[i,"TypeOfProperty"] = "villa"
        elif( ("ård" in house_data_frame.loc[i,"TypeOfProperty"]) == True ):
            house_data_frame.loc[i,"TypeOfProperty"] = "Gård"
        elif( ("hus" in house_data_frame.loc[i,"TypeOfProperty"]) == True ):
            house_data_frame.loc[i,"TypeOfProperty"] = "Fritidshus"
        else:
            house_data_frame.loc[i,"TypeOfProperty"] = "other"
    else:
        house_data_frame.loc[i,"TypeOfProperty"] = "other"

            


In [20]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    print(house_data_frame)

                                                   Name   Size   Room  \
0                                    Björkbacksvägen 53   89.5    3.0   
1                        Wollmar yxkullsgatan 18, vån 3   72.0    3.0   
2                                      Vätterosvägen 15  158.0    7.0   
3                                  Stagneliusgatan 22 C   63.5    2.0   
4                                Södra Bangårdsgatan 4A  101.0    3.0   
5                                         Järpstigen 17  117.0    5.0   
6                                         Fagottvägen 6  117.0    4.0   
7                                    Järsnäs Norrgård 5   60.0    1.0   
8                                     Nordhemsgatan 18A   70.0    2.0   
9                                      Alingsåsvägen 88  162.0    6.0   
10                                     Upplandsgatan 22   52.0    2.0   
11                                       Strandliden 41   79.0    3.0   
12                                       Mimersväge

In [ ]:
house_data_frame.to_csv(r"C:\Users\akash\Documents\AI Sweden\Learning and Resources\Week 2\housing_clean.csv", encoding='latin')